In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import os; import re

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
from sklearn.cross_validation import KFold

In [6]:
from sklearn.grid_search import GridSearchCV

In [7]:
import time
import datetime

In [8]:
from sklearn.linear_model import LogisticRegressionCV

In [38]:
from sklearn.linear_model import LogisticRegression

In [9]:
from sklearn.preprocessing import StandardScaler

In [37]:
from sklearn.grid_search import GridSearchCV

In [139]:
from sklearn.metrics import roc_auc_score

In [10]:
features = pd.read_csv('./_features/features.csv', index_col='match_id')

In [140]:
Xtest = pd.read_csv('./_features_test/features_test.csv', index_col='match_id')

###### drop final attributes

In [11]:
X = features.drop(features.columns[-6:],1)

###### found skips

In [12]:
counts=X.count()

In [13]:
skips=counts[counts.values < 97230].keys()

###### replace na items to ZERO

In [14]:
X_na = X.fillna(0)

###### target var - radiant_win

In [15]:
y=features.radiant_win

###### create crossval generator

In [16]:
cv = KFold(y.size,shuffle=True, n_folds=5)

###### drop categorial and hero features

In [17]:
keys=list(X.keys()[pd.Series(X.keys()).str.contains('hero', na=False)])

keys=keys+['lobby_type','start_time']+X.keys()[pd.Series(X.keys()).str.contains('first_blood_player', na=False)].tolist()

Xdr = X_na.drop(keys,1)

###### transform scale

In [20]:
scdr=StandardScaler()

Xsdr=scdr.fit_transform(Xdr)

###### dummy heroes features

In [21]:
N=112

In [22]:
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in xrange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

###### concat scale and dummy hero features

In [29]:
XX=np.column_stack((Xsdr,X_pick))

In [131]:
np.linspace(1,15,55)

array([  1.        ,   1.25925926,   1.51851852,   1.77777778,
         2.03703704,   2.2962963 ,   2.55555556,   2.81481481,
         3.07407407,   3.33333333,   3.59259259,   3.85185185,
         4.11111111,   4.37037037,   4.62962963,   4.88888889,
         5.14814815,   5.40740741,   5.66666667,   5.92592593,
         6.18518519,   6.44444444,   6.7037037 ,   6.96296296,
         7.22222222,   7.48148148,   7.74074074,   8.        ,
         8.25925926,   8.51851852,   8.77777778,   9.03703704,
         9.2962963 ,   9.55555556,   9.81481481,  10.07407407,
        10.33333333,  10.59259259,  10.85185185,  11.11111111,
        11.37037037,  11.62962963,  11.88888889,  12.14814815,
        12.40740741,  12.66666667,  12.92592593,  13.18518519,
        13.44444444,  13.7037037 ,  13.96296296,  14.22222222,
        14.48148148,  14.74074074,  15.        ])

In [135]:
params={'C':[0.07196857]}

In [136]:
grid = GridSearchCV(LogisticRegression(),cv=cv,param_grid=params,n_jobs=-1,scoring='roc_auc')

In [137]:
grid.fit(XX,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1, param_grid={'C': [0.07196857]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [138]:
grid.grid_scores_

[mean: 0.75159, std: 0.00321, params: {'C': 0.07196857}]

In [ ]:
grid.best_params_

In [97]:
grid.best_score_

0.75159597793923372

In [66]:
grid.best_score_

0.75159475418095012

###### create and fit estimator, with cross validation

In [30]:
estD = LogisticRegressionCV(Cs=15,cv=cv,scoring='roc_auc',n_jobs=-1)

In [31]:
start_time = datetime.datetime.now()
estD.fit(XX,y)
print 'Time elapsed:', datetime.datetime.now() - start_time    

Time elapsed: 0:00:50.683000


In [55]:
estD.scores_[1].max()

0.75593334576875315

In [1]:
estD.C_

NameError: name 'estD' is not defined

In [79]:
for i in estD.scores_[1]:
    print i.mean()

0.745197546514
0.751253687716
0.745080321748
0.752875982063
0.748778480268


In [42]:
estL=LogisticRegression(C=0.07196857)

In [43]:
estL.fit(XX,y)

LogisticRegression(C=0.07196857, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [142]:
Xt = Xtest.drop(Xtest.columns[-6:],1)

In [143]:
counts=Xt.count()

In [144]:
Xt_na = Xt.fillna(0)

In [149]:
tkeys=keys+['lobby_type','start_time']+Xt.keys()[pd.Series(Xt.keys()).str.contains('first_blood_player', na=False)].tolist()

In [157]:
Xtest[tkeys]

,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero,lobby_type,start_time,first_blood_player1,first_blood_player2,lobby_type,start_time,first_blood_player1,first_blood_player2
match_id,,,,,,,,,,,,,,,,,,
6,93,102,52,53,57,26,89,20,29,39,0,1430287923,2,NaN,0,1430287923,2,NaN
7,20,6,65,92,43,54,34,93,99,5,1,1430293357,9,NaN,1,1430293357,9,NaN
10,112,26,29,11,41,91,22,10,49,54,1,1430301774,9,NaN,1,1430301774,9,NaN
13,27,91,74,52,2,110,25,77,62,49,1,1430323933,3,NaN,1,1430323933,3,NaN
16,39,93,30,85,102,14,64,70,3,96,1,1430331112,5,NaN,1,1430331112,5,NaN
18,97,72,31,9,63,59,43,99,5,7,0,1430334264,2,NaN,0,1430334264,2,NaN
19,97,93,50,51,112,20,85,67,35,101,1,1430334995,6,NaN,1,1430334995,6,NaN
24,106,97,26,99,91,46,52,12,7,30,1,1430349466,0,NaN,1,1430349466,0,NaN
33,57,39,33,48,2,54,87,34,30,29,1,1430366484,7,NaN,1,1430366484,7,NaN


In [146]:
tkeys=list(Xt.keys()[pd.Series(X.keys()).str.contains('hero', na=False)])

tkeys=keys+['lobby_type','start_time']+Xt.keys()[pd.Series(Xt.keys()).str.contains('first_blood_player', na=False)].tolist()

Xtdr = Xt_na.drop(tkeys,1)

C:\Users\2\Anaconda2\lib\site-packages\pandas\core\index.py:1160: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 96 but corresponding boolean dimension is 102
  result = getitem(key)


In [ ]:
tscdr=StandardScaler()

Xtsdr=tscdr.fit_transform(Xtdr)

In [ ]:
Xt_pick = np.zeros((Xt.shape[0], N))

for i, match_id in enumerate(Xt.index):
    for p in xrange(5):
        Xt_pick[i, Xt.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        Xt_pick[i, Xt.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [ ]:
XXt=np.column_stack((Xtsdr,Xt_pick))